In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

In [3]:
df = pd.read_csv('/content/drive/MyDrive/집교 2_Team P/user-track-listen_count_filtered5.csv')

In [4]:
# Pickle 파일 읽기
with open('/content/drive/MyDrive/집교 2_Team P/lyrics_Embedding/all_robert-no_stopwords_full.pkl', 'rb') as file:
    data = pickle.load(file)

# DataFrame으로 변환
df_embedding = pd.DataFrame(data, columns=['embedding', 'track_id'])

# track_id를 정수로 변환 (필요하다면)
df_embedding['track_id'] = df_embedding['track_id'].astype(int)

# 'embedding' 열을 768차원의 각 차원으로 나누기
# df_embedding[['embedding_{}'.format(i) for i in range(768)]] = pd.DataFrame(df_embedding['embedding'].tolist(), index=df_embedding.index)

# 'embedding' 열 삭제
# df_embedding = df_embedding.drop(['embedding'], axis=1)

# DataFrame 확인
print(df_embedding.head())


                                           embedding  track_id
0  [-0.086196855, 0.049634214, -0.012802368, -0.1...         2
1  [-0.06731685, 0.044081643, -0.013979832, -0.07...         8
2  [-0.06261022, 0.033318404, -0.0055058626, -0.0...      1524
3  [-0.06850555, 0.028055917, 0.009769377, -0.119...      1785
4  [-0.07563897, 0.015239982, -0.015002853, -0.09...      1787


In [5]:
print(df.shape)
df = pd.merge(df, df_embedding, on='track_id', how='inner')
df.shape

(4645010, 4)


(4644051, 5)

In [6]:
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
track_encoder = LabelEncoder()
lyrics_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user_id'])
df['track_id'] = track_encoder.fit_transform(df['track_id'])
df_embedding['track_id'] = lyrics_encoder.fit_transform(df_embedding['track_id'])

In [7]:
lyrics_dict = dict(zip(df_embedding['track_id'], df_embedding['embedding']))

In [8]:
# !pip install torch torchvision -U

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from tqdm import tqdm

# 데이터 불러오기 (예시: CSV 파일)
# Label Encoding
# PyTorch DataLoader에 맞게 데이터 변환
def df_to_tensor(dataset):
    users = torch.tensor(dataset['user_id'].values, dtype=torch.int)
    items = torch.tensor(dataset['track_id'].values, dtype=torch.int)
    ratings = torch.tensor(dataset['listen_count_bin'].values, dtype=torch.float)
    lyrics_embeddings = torch.tensor(np.vstack(dataset['embedding'].values), dtype=torch.float)
    return users, items, ratings, lyrics_embeddings

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(test_df,test_size=0.5, random_state=42)

train_users, train_items, train_ratings, train_lyrics_embeddings = df_to_tensor(train_df)
val_users,val_items,val_ratings,val_lyrics_embeddings = df_to_tensor(val_df)
test_users, test_items, test_ratings,test_lyrics_embeddings = df_to_tensor(test_df)


train_data = TensorDataset(train_users, train_items, train_ratings,train_lyrics_embeddings)
val_data = TensorDataset(val_users,val_items,val_ratings,val_lyrics_embeddings)
test_data = TensorDataset(test_users, test_items, test_ratings,test_lyrics_embeddings)

train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
val_loader = DataLoader(val_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)
num_users = (df['user_id'].nunique())
num_items = (df['track_id'].nunique())
print(num_users)
print(num_items)

23761
28309


In [9]:
# NCF 모델 정의
class NCF_embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(NCF_embedding, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.lyrics_embedding = nn.Linear(768, embedding_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(embedding_size * 3, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, int(embedding_size/2)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/2), int(embedding_size/4)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/4), int(embedding_size/8)),
            nn.ReLU(),
            nn.Linear(int(embedding_size/8), 1)
        )

    def forward(self, user, item, lyrics_embedding):
        user_embedding = self.user_embedding(user)
        item_embedding = self.item_embedding(item)
        lyrics_embedding = self.lyrics_embedding(lyrics_embedding.reshape(lyrics_embedding.shape[0],768))
        x = torch.cat((user_embedding, item_embedding,lyrics_embedding), dim=1)
        x = self.fc_layers(x)
        return x

# CUDA 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
def train_embedding(embedding,n_epoch):
  # NCF 모델 정의 및 GPU로 이동
  model = NCF_embedding(num_users=num_users, num_items=num_items, embedding_size=embedding)
  model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  # tqdm을 사용하여 학습 및 테스트 진행 상황 확인
  num_epochs = n_epoch
  min_loss = 100
  cnt = 0
  for epoch in range(num_epochs):
      model.train()
      total_loss = 0
      for user, item, rating,embedding in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
          optimizer.zero_grad()
          user, item, rating,embedding = user.to(device), item.to(device), rating.to(device),embedding.to(device)  # GPU로 이동
          output = model(user, item,embedding)
          loss = criterion(output, rating.unsqueeze(1))
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      avg_loss = total_loss / len(train_loader)
      print(f'Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}')

     # Validation
      model.eval()
      total_val_loss = 0
      all_predictions = []
      with torch.no_grad():
          for user, item, rating, embedding in tqdm(val_loader, desc=f'Validation Epoch {epoch+1}'):
              user, item, rating, embedding = user.to(device), item.to(device), rating.to(device), embedding.to(device)  # GPU로 이동
              output = model(user, item, embedding)
              val_loss = criterion(output, rating.unsqueeze(1))
              total_val_loss += val_loss.item()
              all_predictions.append(output)

      # RMSE 계산
      predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
      rmse = np.sqrt(mean_squared_error(val_df['listen_count_bin'].values, predictions))
      avg_val_loss = total_val_loss / len(val_loader)
      print(f'Validation Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_val_loss:.4f}')
      print(f'Validation Epoch {epoch+1}/{num_epochs}, RMSE on validation set: {rmse}')
      if avg_val_loss < min_loss:
          min_loss = avg_val_loss
          cnt = 0
      else:
          cnt += 1
      if cnt == 2:
        print("val_loss did not decrease")
        break

  # Test
  model.eval()
  all_predictions = []
  with torch.no_grad():
      for user, item, _ , embedding in tqdm(test_loader, desc=f'Testing'):
          user, item, embedding = user.to(device), item.to(device), embedding.to(device)  # GPU로 이동
          output = model(user, item,embedding)
          all_predictions.append(output)

  # RMSE 계산
  predictions = torch.cat(all_predictions).squeeze().cpu().numpy()  # CPU로 이동 후 numpy로 변환
  rmse = np.sqrt(mean_squared_error(test_df['listen_count_bin'].values, predictions))
  print(f'Final RMSE on test set: {rmse}')

Using device: cuda


In [10]:
train_embedding(64,15)

Epoch 1/15: 100%|██████████| 12699/12699 [01:31<00:00, 138.34it/s]


Epoch 1/15, Avg. Loss: 1.4231


Validation Epoch 1: 100%|██████████| 2722/2722 [00:12<00:00, 210.24it/s]


Validation Epoch 1/15, Avg. Loss: 1.3391
Validation Epoch 1/15, RMSE on validation set: 1.3857261268335772


Epoch 2/15: 100%|██████████| 12699/12699 [01:31<00:00, 138.07it/s]


...

Epoch 11/15, Avg. Loss: 1.1574


Validation Epoch 11: 100%|██████████| 2722/2722 [00:12<00:00, 213.00it/s]


Validation Epoch 11/15, Avg. Loss: 1.3149
Validation Epoch 11/15, RMSE on validation set: 1.4557175298603842
val_loss did not decrease


Testing: 100%|██████████| 2722/2722 [00:12<00:00, 225.62it/s]


Final RMSE on test set: 1.1459608389706608


In [11]:
train_embedding(256,15)

Epoch 1/15: 100%|██████████| 12699/12699 [02:14<00:00, 94.65it/s]


Epoch 1/15, Avg. Loss: 1.4018


Validation Epoch 1: 100%|██████████| 2722/2722 [00:12<00:00, 211.20it/s]


Validation Epoch 1/15, Avg. Loss: 1.3352
Validation Epoch 1/15, RMSE on validation set: 1.382868623575609


Epoch 2/15: 100%|██████████| 12699/12699 [02:16<00:00, 93.31it/s]


...

Epoch 6/15, Avg. Loss: 1.1122


Validation Epoch 6: 100%|██████████| 2722/2722 [00:12<00:00, 221.95it/s]


Validation Epoch 6/15, Avg. Loss: 1.3165
Validation Epoch 6/15, RMSE on validation set: 1.4900204951872928
val_loss did not decrease


Testing: 100%|██████████| 2722/2722 [00:11<00:00, 233.51it/s]

Final RMSE on test set: 1.1472498750561106


In [12]:
train_embedding(512,15)

Epoch 1/15: 100%|██████████| 12699/12699 [03:09<00:00, 67.09it/s]


Epoch 1/15, Avg. Loss: 1.3952


Validation Epoch 1: 100%|██████████| 2722/2722 [00:12<00:00, 220.59it/s]


Validation Epoch 1/15, Avg. Loss: 1.3407
Validation Epoch 1/15, RMSE on validation set: 1.3760151200885413


Epoch 2/15: 100%|██████████| 12699/12699 [03:12<00:00, 65.98it/s]


...

Epoch 7/15, Avg. Loss: 0.9976


Validation Epoch 7: 100%|██████████| 2722/2722 [00:12<00:00, 212.99it/s]


Validation Epoch 7/15, Avg. Loss: 1.2846
Validation Epoch 7/15, RMSE on validation set: 1.4707120254420363
val_loss did not decrease


Testing: 100%|██████████| 2722/2722 [00:11<00:00, 232.54it/s]


Final RMSE on test set: 1.1343856393663956


In [13]:
train_embedding(768,15)

Epoch 1/15: 100%|██████████| 12699/12699 [04:17<00:00, 49.40it/s]


Epoch 1/15, Avg. Loss: 1.3926


Validation Epoch 1: 100%|██████████| 2722/2722 [00:13<00:00, 205.40it/s]


Validation Epoch 1/15, Avg. Loss: 1.3377
Validation Epoch 1/15, RMSE on validation set: 1.4015203067410813


Epoch 2/15: 100%|██████████| 12699/12699 [04:21<00:00, 48.56it/s]


...

Epoch 7/15, Avg. Loss: 0.9746


Validation Epoch 7: 100%|██████████| 2722/2722 [00:13<00:00, 206.38it/s]


Validation Epoch 7/15, Avg. Loss: 1.2816
Validation Epoch 7/15, RMSE on validation set: 1.5017012262246112
val_loss did not decrease


Testing: 100%|██████████| 2722/2722 [00:11<00:00, 238.25it/s]

Final RMSE on test set: 1.131630069582476
